In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from catboost import CatBoostRegressor
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/raw/train.csv', parse_dates=['Date'])
sample_submission = pd.read_csv('data/raw/sample_submission.csv')

In [4]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
train['Month'] = train['Date'].dt.to_period('M').astype(str)

In [5]:
monthly_agg = (
    train
    .groupby(['Company_ID', 'Product_ID', 'Month'], as_index=False)
    .agg({'Target': 'sum'})
    .rename(columns={'Target': 'sum_sales'})
    .sort_values(by=['Company_ID', 'Product_ID', 'Month'])
)

In [6]:
d = {'Company_ID': [], 'Product_ID': [], 'Month': []}
ar = monthly_agg['Company_ID'].sort_values().unique()
br = monthly_agg['Product_ID'].sort_values().unique()
cr = monthly_agg['Month'].sort_values().unique()
for company_id in ar:
    for product_id in br:
        for month in cr:
            d['Company_ID'].append(company_id)
            d['Product_ID'].append(product_id)
            d['Month'].append(month)
d = pd.DataFrame(d)
monthly_agg = d.merge(monthly_agg, 'left', on=['Company_ID', 'Product_ID', 'Month']).fillna(0)

In [7]:
monthly_agg_saved = monthly_agg.copy()
monthly_agg

,Company_ID,Product_ID,Month,sum_sales
0,0,0,2019-01,0.0
1,0,0,2019-02,0.0
2,0,0,2019-03,0.0
3,0,0,2019-04,0.0
4,0,0,2019-05,0.0
...,...,...,...,...
3459115,3,14668,2023-08,0.0
3459116,3,14668,2023-09,0.0
3459117,3,14668,2023-10,0.0
3459118,3,14668,2023-11,0.0


In [8]:
monthly_agg['Month_date'] = pd.to_datetime(monthly_agg['Month'], format='%Y-%m')

n_lags = 15

subset = []
gb = monthly_agg.groupby(['Company_ID','Product_ID'])
for i in range(n_lags + 1):
    monthly_agg[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
    subset.append(f'prev_sum_sales_{i}')
    if i > 0:
        feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
        monthly_agg[f'rolling_sum_sales_{i + 1}'] = monthly_agg[feats_to_mean].mean(skipna=False, axis=1)
        subset.append(f'rolling_sum_sales_{i + 1}')
        monthly_agg[f'delta_sum_sales_{i}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'prev_sum_sales_{i}']
        subset.append(f'delta_sum_sales_{i}')

monthly_agg['target'] = gb['sum_sales'].shift(-1)

In [9]:
monthly_agg.dropna(subset=['target'], inplace=True)

In [10]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales,Month_date,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_sum_sales_1,prev_sum_sales_2,...,prev_sum_sales_13,rolling_sum_sales_14,delta_sum_sales_13,prev_sum_sales_14,rolling_sum_sales_15,delta_sum_sales_14,prev_sum_sales_15,rolling_sum_sales_16,delta_sum_sales_15,target
0,0,0,2019-01,0.0,2019-01-01,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,0,0,2019-02,0.0,2019-02-01,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,0,0,2019-03,0.0,2019-03-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0,0,2019-04,0.0,2019-04-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0,0,2019-05,0.0,2019-05-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459114,3,14668,2023-07,0.0,2023-07-01,0.0,0.0,0.0,0.0,5.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0
3459115,3,14668,2023-08,0.0,2023-08-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0
3459116,3,14668,2023-09,0.0,2023-09-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0
3459117,3,14668,2023-10,0.0,2023-10-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0


In [11]:
# Сортируем по Month_date (по возрастанию времени)
monthly_agg.sort_values(by=['Month_date'], inplace=True)

# Пример: берём валидацию на последние 3 месяца, которые у нас есть в данных
all_months = sorted(monthly_agg['Month_date'].unique())
N_valid_months = 3
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = monthly_agg[monthly_agg['Month_date'] < valid_threshold].sample(frac=1, random_state=42)
valid_data = monthly_agg[monthly_agg['Month_date'] >= valid_threshold]

In [12]:
feature_cols = subset
cat_cols = ['Company_ID', 'Product_ID']

X_train = train_data[train_data['target'] != 0][feature_cols + cat_cols]
y_train = train_data[train_data['target'] != 0]['target']

X_valid = valid_data[valid_data['target'] != 0][feature_cols + cat_cols]
y_valid = valid_data[valid_data['target'] != 0]['target']

# Укажем, какие столбцы считать категориальными для CatBoost
# (CatBoost умеет работать с ними напрямую, выучивая таргет-кодирование)
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]  # индексы категориальных фич

In [13]:
X_train

,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_sum_sales_1,prev_sum_sales_2,rolling_sum_sales_3,delta_sum_sales_2,prev_sum_sales_3,rolling_sum_sales_4,delta_sum_sales_3,...,rolling_sum_sales_14,delta_sum_sales_13,prev_sum_sales_14,rolling_sum_sales_15,delta_sum_sales_14,prev_sum_sales_15,rolling_sum_sales_16,delta_sum_sales_15,Company_ID,Product_ID
29490,0.0,2.0,1.0,-2.0,0.0,0.666667,0.0,0.0,0.50,0.0,...,1.500000,-6.0,3.0,1.600000,-3.0,0.0,1.5000,0.0,0,499
1229635,1.0,1.0,1.0,0.0,0.0,0.666667,1.0,2.0,1.00,-1.0,...,0.857143,-1.0,0.0,0.800000,1.0,2.0,0.8750,-1.0,1,6182
1560952,3.0,3.0,3.0,0.0,2.0,2.666667,1.0,2.0,2.50,1.0,...,1.214286,3.0,0.0,1.133333,3.0,2.0,1.1875,1.0,1,11802
1724800,1.0,1.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.50,1.0,...,0.642857,-1.0,1.0,0.666667,0.0,0.0,0.6250,1.0,1,14588
3174609,1.0,0.0,0.5,1.0,0.0,0.333333,1.0,1.0,0.50,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,9840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970545,3.0,2.0,2.5,1.0,1.0,2.000000,2.0,0.0,1.50,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6375
2099764,7.0,6.0,6.5,1.0,25.0,12.666667,-18.0,18.0,14.00,-11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,6274
1385092,1.0,0.0,0.5,1.0,0.0,0.333333,1.0,0.0,0.25,1.0,...,43.642857,1.0,125.0,49.066667,-124.0,2.0,46.1250,-1.0,1,8828
2595552,7.0,3.0,5.0,4.0,0.0,3.333333,7.0,7.0,4.25,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,21


In [14]:
model = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.015,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=42,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=100
)

0:	learn: 43.7392103	test: 48.4552474	best: 48.4552474 (0)	total: 536ms	remaining: 13m 23s
100:	learn: 28.2593925	test: 33.0440445	best: 33.0440445 (100)	total: 34s	remaining: 7m 51s
200:	learn: 23.6445956	test: 27.8268864	best: 27.8268864 (200)	total: 1m 2s	remaining: 6m 45s
300:	learn: 22.4169304	test: 26.1885441	best: 26.1885441 (300)	total: 1m 33s	remaining: 6m 12s
400:	learn: 21.9303985	test: 25.5765892	best: 25.5765892 (400)	total: 2m 3s	remaining: 5m 39s
500:	learn: 21.6559954	test: 25.2448744	best: 25.2448744 (500)	total: 2m 35s	remaining: 5m 9s
600:	learn: 21.5555620	test: 25.1313187	best: 25.1313187 (600)	total: 3m 3s	remaining: 4m 33s
700:	learn: 21.4307427	test: 25.0115572	best: 25.0115572 (700)	total: 3m 32s	remaining: 4m 1s
800:	learn: 21.2974999	test: 24.8678617	best: 24.8678617 (800)	total: 4m 1s	remaining: 3m 30s
900:	learn: 21.1511871	test: 24.7130911	best: 24.7130911 (900)	total: 4m 32s	remaining: 3m 1s
1000:	learn: 21.0636822	test: 24.6393933	best: 24.6393933 (1000)

In [15]:
from sklearn.metrics import mean_absolute_error
scale_factor = (monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12']
                .groupby(['Company_ID', 'Product_ID'])['sum_sales']
                .agg(lambda x: int(x.sum() > 0)).sum() * 3) / len(sample_submission)

y_pred_valid = model.predict(X_valid)
mae_valid = mean_absolute_error(y_valid, y_pred_valid)
print("Raw MAE (valid):    ", mae_valid)
print("Scaled MAE (valid): ", mae_valid * scale_factor)

Raw MAE (valid):     23.5758640648635
Scaled MAE (valid):  9.609997288244054


In [16]:
def predict_next_month(df, model):
    df = df.copy()
    df['Month_date'] = pd.to_datetime(df['Month'], format='%Y-%m')
    df['Month_pred'] = (df['Month_date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').astype(str)
    df = df.sort_values(by='Month_date')

    gb = df.groupby(['Company_ID', 'Product_ID'])
    for i in range(n_lags + 1):
        df[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
        if i > 0:
            feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
            df[f'rolling_sum_sales_{i + 1}'] = df[feats_to_mean].mean(skipna=False, axis=1)
            df[f'delta_sum_sales_{i}'] = df['prev_sum_sales_0'] - df[f'prev_sum_sales_{i}']

    df['Target'] = gb['sum_sales'].shift(-1)

    to_predict = df['Target'].isna()
    df.loc[to_predict, 'Target'] = model.predict(df.loc[to_predict, feature_cols + cat_cols])
    return df.loc[to_predict, ['Company_ID', 'Product_ID', 'Month_pred', 'Target']]

In [17]:
mag = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') < valid_threshold].copy()
preds = []
for i in range(N_valid_months):
    p = predict_next_month(mag, model)
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    preds.append(p)
    mag = pd.concat([mag, p], axis=0)

preds = pd.concat(preds, axis=0)
valid_data = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') >= valid_threshold].copy()
merged = valid_data.merge(preds, on=['Company_ID', 'Product_ID', 'Month'], suffixes=('_actual', '_pred'))
raw_mae = mean_absolute_error(merged['sum_sales_actual'], merged['sum_sales_pred'])
print("Рекурсивный прогноз - raw MAE (valid):    ", raw_mae)
print("Рекурсивный прогноз - scaled MAE (valid): ", raw_mae * scale_factor)

Рекурсивный прогноз - raw MAE (valid):     28.61077701644865
Рекурсивный прогноз - scaled MAE (valid):  11.662329269721244


In [18]:
full_train = monthly_agg.dropna(subset=['target']).sample(frac=1, random_state=42)
X_full = full_train[full_train['target'] != 0][feature_cols + cat_cols]
y_full = full_train[full_train['target'] != 0]['target']

final_model = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.015,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=42,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

0:	learn: 43.9175999	total: 430ms	remaining: 10m 44s
100:	learn: 28.3251306	total: 37.4s	remaining: 8m 38s
200:	learn: 23.8020968	total: 1m 6s	remaining: 7m 12s
300:	learn: 22.5370276	total: 1m 38s	remaining: 6m 32s
400:	learn: 22.0361089	total: 2m 12s	remaining: 6m 3s
500:	learn: 21.7273008	total: 2m 46s	remaining: 5m 32s
600:	learn: 21.5881502	total: 3m 21s	remaining: 5m
700:	learn: 21.4856037	total: 3m 55s	remaining: 4m 28s
800:	learn: 21.4218903	total: 4m 23s	remaining: 3m 49s
900:	learn: 21.3221286	total: 4m 52s	remaining: 3m 14s
1000:	learn: 21.1670391	total: 5m 23s	remaining: 2m 41s
1100:	learn: 20.2447620	total: 5m 50s	remaining: 2m 7s
1200:	learn: 19.7122958	total: 6m 21s	remaining: 1m 35s
1300:	learn: 19.4258277	total: 6m 49s	remaining: 1m 2s
1400:	learn: 19.1270976	total: 7m 16s	remaining: 30.8s
1499:	learn: 18.7230113	total: 7m 43s	remaining: 0us


In [19]:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[1]).astype(int)
sample_submission['Month_pred'] = sample_submission['Id'].apply(lambda x: x.split('_')[2]).astype(str)
sample_submission

,Id,Target,Company_ID,Product_ID,Month_pred
0,0_1_2024-01,0,0,1,2024-01
1,0_2_2024-01,0,0,2,2024-01
2,0_3_2024-01,0,0,3,2024-01
3,0_5_2024-01,0,0,5,2024-01
4,0_6_2024-01,0,0,6,2024-01
...,...,...,...,...,...
113851,3_14664_2024-03,0,3,14664,2024-03
113852,3_14665_2024-03,0,3,14665,2024-03
113853,3_14666_2024-03,0,3,14666,2024-03
113854,3_14667_2024-03,0,3,14667,2024-03


In [20]:
preds = []
mag = monthly_agg_saved.copy()
for i in range(3):
    p = predict_next_month(mag, final_model)
    preds.append(p.copy())
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    mag = pd.concat([mag, p], axis=0)
preds = pd.concat(preds, axis=0)
sample_submission = sample_submission.drop(columns=['Target'], axis=1).merge(preds, 'left', on=['Company_ID', 'Product_ID', 'Month_pred'])

In [21]:
to_mult = monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12'].groupby(['Company_ID', 'Product_ID'])['sum_sales'].agg(lambda x: int(x.sum() > 0)).reset_index()
for i in tqdm.tqdm(to_mult.index):
    sample_submission.loc[(sample_submission['Company_ID'] == to_mult.loc[i, 'Company_ID']) &
                          (sample_submission['Product_ID'] == to_mult.loc[i, 'Product_ID']), 'Target'] *= to_mult.loc[i, 'sum_sales']

100%|██████████| 57652/57652 [00:34<00:00, 1668.88it/s]


In [22]:
sample_submission[['Id', 'Target']]

,Id,Target
0,0_1_2024-01,5.567015
1,0_2_2024-01,0.000000
2,0_3_2024-01,0.000000
3,0_5_2024-01,2.009497
4,0_6_2024-01,2.780513
...,...,...
113851,3_14664_2024-03,0.000000
113852,3_14665_2024-03,180.101063
113853,3_14666_2024-03,0.000000
113854,3_14667_2024-03,1.044024


In [23]:
sample_submission[['Id', 'Target']].to_csv('data/submissions/bruh.csv', index=False)